In [1]:
# -*- coding: utf-8 -*-
%matplotlib qt5
import numpy as np
from astropy.io import fits
from astropy.table import Table
import matplotlib.pyplot as plt
import photometry_utils as phot

# PSF Photometry
### ! This workflow is designed to be executed one task after the other. Wait for the cells to be finished before running the next one, otherwise passed variables may be empty. DO NOT RUN THE COMPLETE NOTEBOOK AT ONCE ! 

## 0. Definition of fit parameter

In [2]:
filter_name = 'V'
zeropoint = 25.24
psf_size = 20

## 1. Loading the data with astropy

In [8]:
image = fits.getdata(f'../cutout_BS90_{filter_name}.fits')

## 2. Select the stars to determine the PSF
### 2.1. Choose stars from the image

In [9]:
psf_stars = phot.select_psf_stars_from_image(image)

C:\ProgramData\Anaconda3\lib\site-packages\matplotlib\image.py:479: RuntimeWarning: invalid value encountered in subtract
  A_scaled -= a_min


### 2.2. Evaluate choosen stars

In [10]:
peaks_tbl, psf_stamps = phot.create_psf_stamps(image, psf_stars, psf_size)
selected_stars_idx = phot.evaluate_selected_stars(psf_stamps, psf_size)

In [11]:
good_stars = phot.mask_stars(peaks_tbl, selected_stars_idx, psf_size, image.shape, filter_name)

or load an existing list:

In [4]:
good_stars = Table.read(f'selected_stars_{filter_name}.dat', format='ascii')

## 3. Build PSF model

In [12]:
bkg_sub_image = phot.subtract_background(image)
psf = phot.determine_psf(bkg_sub_image, good_stars, psf_size)

C:\Users\danie\OneDrive\Desktop\CMD\FP30_Scripts\photometry_utils.py:271: RuntimeWarning: invalid value encountered in subtract
  return image - bkg.background


In [11]:
phot.plot_image(psf.data, min_max_percantage=99.9)

(<Figure size 640x480 with 2 Axes>, <AxesSubplot:>)

## 4. Performing iterative PSF  photometry

In [13]:
results, residuals = phot.do_photometry(bkg_sub_image, psf, threshold=10, niters=2, zeropoint=zeropoint)
results.write(f'photometry_{filter_name}.dat', format='ascii', overwrite=True)

C:\ProgramData\Anaconda3\lib\site-packages\photutils\detection\irafstarfinder.py:456: RuntimeWarning: invalid value encountered in true_divide
  return moments / self.moments[:, 0, 0][:, np.newaxis, np.newaxis]
C:\ProgramData\Anaconda3\lib\site-packages\photutils\detection\irafstarfinder.py:472: RuntimeWarning: invalid value encountered in true_divide
  return np.sqrt(self.mu_diff**2


ValueError: Input position contains invalid values (NaNs or infs).

In [ ]:
print(len(results))
phot.plot_image(residuals)